# Final project for the course of Applied Math: numerical analysis and scientific computing, academic year 2025-26

For any question about the final project, contact the tutor anna.ivagnes@sissa.it

## Preliminary instructions

Please provide the solution to the final project in the format of a Python Jupyter Notebook (`.ipynb`), and (if needed) a `utils.py` file with the functions you have used in the Notebook.

Don't forget to document your code: add comments and explanations useful to let us understand what you have done.

Suggestion: use functions if instructions are repeated more than twice.

You can either create your own fork of the repository on Github and add-commit-push your Notebook there, or send us the Notebook via email before taking the oral examination.

During the oral examination, you could be asked to explain some portions of the provided notebook.

---

## Assignments
### Assignment 1: The discharge cycle of a lithium-ion battery.

NASA provides some databases to study the charge and discharge cycles of a lithium-ion battery.
We provide in CSV format the data containing:
- the time $t$ associated with a discarge cycle (measured in seconds),
- the voltage $V$ associated with the time instances,
- the current $I$ associated with the time instances.
- 
The data are in `data_assignment1/battery_discharge.csv`.

**Tasks**:
1. Read the data in the CSV file using `pandas` and plot the voltage against time.
2. Interpolation task:
    - Estimate the voltage at times 500, 1500, 2500 seconds using **Lagrange interpolation** using different equispaced downsamplings of the data (one point every 5, 10, ecc). This corresponds to varying the number of nodes, and hence, the degree of the fitting polynomial.
    - Compare the results with the built-in scipy function `scipy.interpolate.lagrange`;
    - Extrapolation: if we only use data in the time window $[0, 3100]$, how does the interpolation performs at $t=3300$?
    - **Bonus**: are we able to improve the Lagrange interpolation, considering as data all the data points in $[0, 3100]$ by using other interpolation approaches? Which is the method with the closest prediction at $t=3300$?
    *Hint*: explore the functions in `scipy.interpolate`. 
3. Regression task:
    - Fit the model: $$V(t) = V_0 + a t + b t^2 + ct^3$$
    using data in time interval $[0, 3100]$. Is the least-squares fitting polynomial performing better than the interpolation approaches you have seen?
4. Find the time $t^{\star}$ such that:
    $V(t^{\star}) = 3.6,$ where $V(t)$ is approximated with one of the interpolation techniques you used before.

---


## Assignment 2: Predict Airbnb Prices with Linear Regression From Scratch
Airbnb provides public and downloadable databases at https://insideairbnb.com with the data related to rooms (prices, room type, number of reviews, distance to center, etc).

Choose a city and complete the following tasks.
You can either choose one city in the folder `data_assignment2` or choose another city directly from the database.

**Tasks**:
1. Read the data using `pandas`, and in particular the data related to price, latitude, longitude,
  number of reviews, room type.
2. Analyze the dependency of the price with respect to different variables:
- distance from center $d$; 
- number of reviews $r$;
- room type $t$.

*Hint*: compute the distance from the city center based on the latitude and longitude (you can find the latitude and longitude of a city center online or from Google Maps).

  **2a.** Start from considering the price only as a linear or quadratic function of the distance from the city center $p(d)=c_0+c_1 d$ or $p(d)=c_0 + c_1 d +c_2 d^2$. Build the matrix $A$ and solve $$\min_{c} \|Ac-p\|_2, $$  where $c$ is the vector of coefficients via:
  - normal equations,
  - $QR$ factorization.

  Plot the approximated $p$ as a function of $d$ for different room types.

  **2b.** How is the conditioning of $A^T A$? How can we improve it?

  **2c.** Now consider only the number of reviews, and we want the price only dependent on it, namely $p(r)$. Repeat the same procedure as in (2a). Is your model better approximating the price? Is it better if you consider separately different room types $t$?
  Plot the approximated $p$ as a function of $r$ for different room types.

  *Hint*: compare the models in terms of the residual $\|Ac-p\|_2$.

  *Hint*: consider solving different systems for different room types.

  **2d.** Consider $p \sim p(d, r, t)$ and fit a least squares problem. Is this approximation more accurate than in the cases $p(d)$ and $p(r)$? Plot the approximated and the exact $p$ as a function of $d$ and $r$ for different room types.

**Final remark**: don't forget you are dealing with real and complex data, so it is hard to obtain good results. The important thing is that you justify your choices and the logic behind the solution you provide.

---

## Assignment 3: Image Deblurring via Variational Regularization

Recovering the original image from a blurred observation is an important problem in image processing and scientific computing. This task allows you to explore linear systems, both direct and iterative solvers. In this exercise, you will model the image deblurring problem as a linear system $Ku=f$, where $K$ represents the blur operator, $f$ is the observed blurred (and possibly noisy) image, and $u$ is the unknown sharp image.

**Tasks**:

1. Dataset and Blur Simulation: Load a standard grayscale image (e.g., `skimage.data.camera()`). Apply a simple convolutional blur (e.g., 3×3 or 5×5 Gaussian-like kernel, see code below).

    *Hint*: use functions `skimage.img_as_float` to read the image, and `scipy.signal.convolve2d` to apply the blurring kernel.

2. Linear System Formulation: Represent the blurring operation as a linear system $Ku=f$, where $f$ is the blurred image, and $u$ is the original image we want to reconstruct by solving the system.

    *Hints*:
       - Compute each column $j$-th of matrix $K$ by applying the `convolve2d` operation on a basis vector (which has component $1$ only at column $j$ and $0$ everywhere else).
       - To be precise, $f$ and $u$ are the *vectorized* images (if the image has shape $(N, N)$, the vectorized image has shape $(N^2, 1)$).
   
4. Direct Solver: Solve the linear system using a direct solver ($LU$ factorization).
 
   *Hint*: consider applying methods for sparse matrices (built-in `scipy` methods).
   
5. Iterative Solvers: Implement and test Jacobi and Gauss–Seidel iterative methods.
Optionally, use Conjugate Gradient ($CG$) if the system is symmetric and positive definite.

6. Compare the methods in terms of convergence rates and number of iterations: plot the residual vs iterations, and print the accuracy of the obtained images in terms of the mean squared error with respect to the original one.
   
7. **Bonus**: repeat the same analysis adding also a Gaussian noise to the blurred image $f$. How do the methods perform now?


**Suggestion**: always solve the problems depending on the computational power of your laptop. The image might be too big: for example `skimage.data.camera()` produces an image of size $(512, 512)$, and the matrix $K$ would have size $(512^2, 512^2)$. Consider downsampling it, for example considering an image of size $(64, 64)$ or $(32, 32)$.

In [1]:
import numpy as np

# Strong Gaussian kernel
def gaussian_kernel(size=21, sigma=2.):
    ax = np.arange(-size//2 + 1., size//2 + 1.)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    return kernel / kernel.sum()

kernel = gaussian_kernel(size=21, sigma=1.)